# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Загрузим необходимые библиотеки

In [14]:
import pandas as pd
import numpy as np
import transformers as tf
import re 
import nltk
from tqdm import notebook
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
# Загрузка необходимых ресурсов
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/a1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/a1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/a1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/a1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Загрузим данные

In [3]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('toxic_comments.csv')

Просмотрим общую информацию по данным:

In [4]:
display(df.info())
display(df.head())
display(df.shape)
display(df.describe())
display(df.isnull().sum())
print(f"Количество дубликатов: {df.duplicated().sum()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


(159292, 3)

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Unnamed: 0    0
text          0
toxic         0
dtype: int64

Количество дубликатов: 0


**Вывод:**
- Фрейм состоит из 159292 строк и 3 столбцов
- Пропуски отсутствуют
- Типы данных для столбцов соответствуют
- Явные дубликаты отсутствуют

Необходимо удалить столбец unnamed:0, так как нам не нужен индексный столбец

In [5]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Объявим стоп слова и лемматизацию для дальнейшей функции

In [6]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

Напишем функцию и применим ее

In [7]:
def get_wordnet_pos(word):
    "Преобразует POS-тег в формат, принимаемый WordNetLemmatizer."
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

def pred(text):
    # Уберем из текста все лишнее
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.lower()
    # Токенизируем
    token = nltk.word_tokenize(text)
    # Убираем стоп-слова и лемматизируем с учетом POS-тегов
    text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in token if word not in stop_words]
    text = ' '.join(text)
    return text

# Применение функции
df['pred'] = df['text'].apply(pred)

Просмотрим таблицу с новым признаком

In [8]:
df.head()

,text,toxic,pred
0,Explanation\nWhy the edits made under my usern...,0,explanation edits make username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really try edit war guy constantly rem...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestion improvement wonder sectio...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page


**Подготовим выборки**

In [9]:
df_1 = df.copy()
y = df_1['toxic']
df_1 = df_1.drop(['toxic'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.3, stratify = y)

Просмотрим размерности выборок

In [10]:
display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(111504, 3)

(47788, 3)

(111504,)

(47788,)

**Вывод:**
- Данные очищены от лишних символов
- Данные лемматизированы и токенизированы
- Данные разбиты на выборки для обучения моделей

## Обучение

Подготовим функцию, которая позволит обучить модели а также вывести лучшие результаты. По условию будем использовать метрику f1

In [11]:
def train(model, params):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(min_df = 1)),
        ('model', model)])
    grid = GridSearchCV(pipeline, cv = 5, n_jobs = 1, param_grid = params ,scoring = 'f1', verbose = False)
    grid.fit(X_train['pred'], y_train)
    print('Лучший результат:', grid.best_score_)
    print('Лучшие параметры:', grid.best_params_)
    return grid

Начнем с логистической регрессии

In [12]:
lr_model = train(LogisticRegression(max_iter = 200), {"model__C":[0.1, 1.0, 10.0], "model__penalty":["l2"]})

Лучший результат: 0.7747885946004529
Лучшие параметры: {'model__C': 10.0, 'model__penalty': 'l2'}


Отлично, значение метрики уже показывает, что модель подходит нам по условию, но просмотрим и другие модели

Модель svc

In [15]:
svc_model = train(LinearSVC(max_iter=500),{"model__C": [0.1,1.0,10.0],"model__penalty": ['l1', 'l2']}) 

Лучший результат: 0.7820610072388219
Лучшие параметры: {'model__C': 1.0, 'model__penalty': 'l1'}


Модель DesicionTreeClassifier

In [16]:
tree_model = train(DecisionTreeClassifier(), {'model__max_depth':[2,4,6],"model__min_samples_split": [2, 5, 10],"model__min_samples_leaf": [1, 2, 4]})

Лучший результат: 0.561447971235285
Лучшие параметры: {'model__max_depth': 6, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2}


И наконец модель CatBoost

In [17]:
cat_model = train(CatBoostClassifier(), {'model__depth': [4,6],'model__iterations': [50]})

Learning rate set to 0.5
0:	learn: 0.3483768	total: 240ms	remaining: 11.7s
1:	learn: 0.2661854	total: 404ms	remaining: 9.7s
2:	learn: 0.2414539	total: 577ms	remaining: 9.04s
3:	learn: 0.2302306	total: 731ms	remaining: 8.4s
4:	learn: 0.2223362	total: 898ms	remaining: 8.08s
5:	learn: 0.2137399	total: 1.06s	remaining: 7.79s
6:	learn: 0.2079154	total: 1.23s	remaining: 7.56s
7:	learn: 0.2030888	total: 1.39s	remaining: 7.27s
8:	learn: 0.1993709	total: 1.55s	remaining: 7.04s
9:	learn: 0.1964499	total: 1.7s	remaining: 6.79s
10:	learn: 0.1932685	total: 1.85s	remaining: 6.57s
11:	learn: 0.1907685	total: 2s	remaining: 6.35s
12:	learn: 0.1861181	total: 2.18s	remaining: 6.2s
13:	learn: 0.1841521	total: 2.33s	remaining: 5.99s
14:	learn: 0.1809358	total: 2.5s	remaining: 5.82s
15:	learn: 0.1788827	total: 2.65s	remaining: 5.63s
16:	learn: 0.1765829	total: 2.81s	remaining: 5.45s
17:	learn: 0.1749371	total: 2.96s	remaining: 5.26s
18:	learn: 0.1728858	total: 3.12s	remaining: 5.08s
19:	learn: 0.1710052	tot

**Вывод:**
* Модель LogisticRegression:
    * Лучший результат: 0.7747885946004529
    * Лучшие параметры: {'model__C': 10.0, 'model__penalty': 'l2'}
* Модель SVC:
    * Лучший результат: 0.7820610072388219
    * Лучшие параметры: {'model__C': 1.0, 'model__penalty': 'l1'}
* Модель DecisionTreeClassifier:
    * Лучший результат: 0.561447971235285
    * Лучшие параметры: {'model__max_depth': 6, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2}
* Модель CatBoost
    * Лучший результат: 0.7255484543456935
    * Лучшие параметры: {'model__depth': 6, 'model__iterations': 50}
* Значение метрик у всех моделей кроме DT подходит по условию задачи, но выберем наилучшую модель для тестирования - SVC

**Тестирование**

In [18]:
res = f1_score(y_test, svc_model.predict(X_test['pred']))
print(res)

0.7908238219007797


Итого модель SVC, показала лучшую метрику на обучении -  0.782, а на тестировании также набрала  0.79, что довольно хорошо

## Выводы

В ходе выполнения проекта были сделаны следующие шаги:
* Загрузили и предобработали данные:
    - Фрейм состоит из 159292 строк и 3 столбцов
    - Пропуски отсутствуют
    - Типы данных для столбцов соответствуют
    - Явные дубликаты отсутствуют
    - Удалили столбец unnamed:0, так как нам не нужен индексный столбец
    - Данные очищены от лишних символов
    - Данные лемматизированы и токенизированы
    - Данные разбиты на выборки для обучения моделей
* Обучили модели:
    -  Написали функцию, которая позволит обучать модели с поиском оптимальных параметров
    -  Получили такие результаты:
        * Модель LogisticRegression:
            * Лучший результат: 0.774574686341823
            * Лучшие параметры: {'model__C': 10.0, 'model__penalty': 'l2'}
        * Модель SVC:
            * Лучший результат: 0.7889807618637914
            * Лучшие параметры: {'model__C': 1.0, 'model__penalty': 'l1'}
        * Модель DecisionTreeClassifier:
            * Лучший результат: 0.5359419182089493
            * Лучшие параметры: {'model__max_depth': 6, 'model__min_samples_leaf': 1, 'model__min_samples_split': 5}
        * Модель CatBoost
            * Лучший результат: 0.7295468508422724
            * Лучшие параметры: {'model__depth': 6, 'model__iterations': 50}
        * Значение метрик у всех моделей кроме DT подходит по условию задачи, но выберем наилучшую модель для тестирования - SVC
    - Протестировали модель SVC и получили значение метрики 0.785 